In [3]:
import numpy as np

class LinearSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        # Initialization
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        # Ensure y values are converted to {-1, 1}
        # Maps 0 to -1, and keeps 1 as 1 (handling the {0, 1} case)
        y_ = np.where(y <= 0, -1, 1)

        # Training Loop
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                # Check the Margin Condition: y_i * (w . x_i - b) >= 1
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1

                if condition:
                    # If True (Correct & Safe):
                    # Update w based ONLY on the regularization term
                    # Gradient of (lambda * ||w||^2) is (2 * lambda * w)
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    # If False (Misclassified or Inside Margin):
                    # Update w based on Regularization + Hinge Loss
                    # Grad w.r.t w: 2 * lambda * w - y_i * x_i
                    self.w -= self.lr * (2 * self.lambda_param * self.w - y_[idx] * x_i)

                    # Update b based on Hinge Loss (Regularization term has no b)
                    # Loss term is: 1 - y_i(w.x_i - b) -> 1 - y_i.w.x_i + y_i.b
                    # Grad w.r.t b is: y_i
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        # Prediction
        # Compute linear output: f(x) = w . X - b
        linear_output = np.dot(X, self.w) - self.b
        # Return the sign (-1 or 1)
        return np.sign(linear_output)

In [5]:
if __name__ == "__main__":
    # Generate simple linearly separable data
    # Class 1: Points around [2, 2]
    # Class -1: Points around [6, 6]
    X = np.array([
        [2.0, 3.0], [1.0, 2.0], [2.0, 1.0],  # Class 1
        [5.0, 6.0], [6.0, 7.0], [7.0, 6.0]   # Class -1
    ])

    # Labels (1 for first class, -1 for second)
    # Note: The code handles {0, 1} automatically, but we use {1, -1} here for clarity
    y = np.array([1, 1, 1, -1, -1, -1])

    # Initialize and Train SVM
    clf = LinearSVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
    clf.fit(X, y)

    # Make Predictions
    predictions = clf.predict(X)

    # Display Results
    print("Training Results")
    print(f"Learned Weights (w): {clf.w}")
    print(f"Learned Bias (b):    {clf.b}")
    print("\nPredictions")
    print(f"Actual Labels:    {y}")
    print(f"Predicted Labels: {predictions.astype(int)}") # cast to int for clean display

    # Check Accuracy
    accuracy = np.mean(y == predictions)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")

    # Test on a new unseen point
    new_point = np.array([[4.0, 4.0]]) # A point roughly in the middle
    pred_new = clf.predict(new_point)
    print(f"\nPrediction for new point {new_point}: {int(pred_new[0])}")

Training Results
Learned Weights (w): [-0.2809791  -0.23619515]
Learned Bias (b):    -1.804999999999912

Predictions
Actual Labels:    [ 1  1  1 -1 -1 -1]
Predicted Labels: [ 1  1  1 -1 -1 -1]

Accuracy: 100.00%

Prediction for new point [[4. 4.]]: -1
